In [ ]:
# default_exp core

In [ ]:
#export
import csv
from gensim import corpora, models, matutils
import matplotlib.pyplot as plt
from medtop.helpers import *
from medtop.nlp_helpers import *
from medtop.preprocessing import *
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
from scipy.cluster.hierarchy import ward, cut_tree
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.manifold import MDS
from sklearn.metrics import silhouette_score, pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from textblob import TextBlob
import umap.umap_ as umap

# Core

> Methods intended to be called publicly

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
#export
def import_docs(path_to_file_list):
    "Imports and processes the list of documents contained in the given file"
    
    # Extract list of files from text document
    with open(path_to_file_list) as file:
        file_list = file.read().strip().split('\n')
    
    # Create a dataframe containing the doc_id, file name, and raw text of each document
    doc_cols = ["doc_id", "file", "text", "tokens"]
    doc_df = pd.DataFrame(columns=doc_cols)
    
    # Create a dataframe containing the doc_sent_id, doc_id, sent_id, raw text, tokens, and PoS tags for each sentence
    sent_cols = ["doc_sent_id", "doc_id", "sent_id", "text", "tokens", "pos_tags"]
    data = pd.DataFrame(columns=sent_cols)
    
    for doc_id, file in enumerate(file_list):
        with open(file, "r") as file_content:
            doc_text = file_content.read()
            
        sent_tokens, sent_pos, raw_sent = tokenize_and_stem(doc_text)
        for sent_id, _ in enumerate(sent_tokens):
            # Populate a row in data for each sentence in the document
            doc_sent_id = f"doc.{doc_id}.sent.{sent_id}"
            sent_text = raw_sent[sent_id]
            tokens = sent_tokens[sent_id]
            pos_tags = sent_pos[sent_id]
            sent_row = pd.Series([doc_sent_id, doc_id, sent_id, sent_text, tokens, pos_tags], index=sent_cols)
            data = data.append(sent_row, ignore_index=True)
        
        # Populate a row in doc_df for each file loaded
        doc_tokens = [token for sent in data[data.doc_id==doc_id].tokens for token in sent]
        doc_row = pd.Series([doc_id, file, doc_text, doc_tokens], index=doc_cols)
        doc_df = doc_df.append(doc_row, ignore_index=True)
    
    return data, doc_df

In [ ]:
#export
def sentences_to_disk(data, file_name = 'output/DocumentSentenceList.txt'):
    "Writes the raw sentences to a file organized by document and sentence number"
    data.to_csv(file_name, columns=["doc_sent_id", "text"], sep='\t', encoding='utf-8', index=False, quoting=csv.QUOTE_NONE, quotechar="",  escapechar="\\")

In [ ]:
#export
def create_tfidf(path_to_seed_topics_file_list, input_doc_df = None):
    "Creates a TF-IDF matrix from the tokens in the seed topics documents and optionally, the input corpus"
    
    # Import seed topics documents
    _, seed_doc_df = import_docs(path_to_seed_topics_file_list)
     
    # Bag of Words (BoW) is a list of all tokens by document
    bow_docs = list(seed_doc_df.tokens)
    
    if input_doc_df is not None:
        bow_docs = bow_docs + list(input_doc_df.tokens)

    # Create a TF-IDF matrix using document tokens and gensim
    dictionary = corpora.Dictionary(bow_docs)
    corpus = [dictionary.doc2bow(text) for text in bow_docs]
    tfidf = models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    num_terms = len(corpus_tfidf.obj.idfs)
    tfidf_dense = matutils.corpus2dense(corpus_tfidf, num_terms)
    print(f"Size of TF-IDF: {len(tfidf_dense[0])} Docs, {len(tfidf_dense)} Tokens")
    
    return tfidf_dense, dictionary

In [ ]:
#hide
# path_to_file_list = 'data/2019.03.11_DevCorpus/file_list.txt'
# data, doc_df = import_docs(path_to_file_list)
# path_to_seed_topics_file_list = 'data/2019.03.12_SEED_TOPICS/FILELIST.txt'
# tfidf, dictionary = create_tfidf(path_to_seed_topics_file_list, doc_df)
# feature_names = dictionary.token2id
# tdm = tfidf
# append_phrase_column(data, feature_names, tdm)
# data

Size of TF-IDF: 14 Docs, 188 Tokens


,doc_sent_id,doc_id,sent_id,text,tokens,pos_tags
0,doc.0.sent.0,0,0,I am currently on my first AI in the Emergency...,"[currently, emergency, department]","[[currently, RB], [Emergency, NNP], [Departmen..."
1,doc.1.sent.0,1,0,I am currently working in the cardiac surgery ...,"[currently, working, cardiac, surgery, icu, gr...","[[currently, RB], [working, VBG], [cardiac, NN..."
2,doc.2.sent.0,2,0,"As many have said, reading these messages whil...","[said, reading, message, starting, rotation, c...","[[said, VBD], [reading, VBG], [messages, NNS],..."
3,doc.3.sent.0,3,0,"Very interesting posts by you guys, I really e...","[interesting, post, guy, enjoyed, reading]","[[interesting, JJ], [posts, NNS], [guys, VBP],..."
4,doc.4.sent.0,4,0,Just have to say that all of you seem to be do...,"[seem, wonderfully, rotation]","[[seem, VBP], [wonderfully, RB], [rotations, N..."
5,doc.5.sent.0,5,0,"I'm currently on the MRICU rotation, after com...","[currently, mricu, rotation, completing, inter...","[[currently, RB], [MRICU, NNP], [rotation, NN]..."
6,doc.6.sent.0,6,0,The greatest (but most fun and rewarding) chal...,"[greatest, fun, rewarding, challenge, faced, t...","[[greatest, JJS], [fun, JJ], [rewarding, VBG],..."
7,doc.7.sent.0,7,0,Hope you're all having a great AI experience s...,"[hope, great, experience, far]","[[Hope, NN], [great, JJ], [experience, NN], [f..."
8,doc.8.sent.0,8,0,"Anyhow, because of the odd nature of my schedu...","[anyhow, odd, nature, schedule, completed, sin...","[[Anyhow, NNP], [odd, JJ], [nature, NN], [sche..."
9,doc.9.sent.0,9,0,I am currently completing my ICU AI requiremen...,"[currently, completing, icu, requirement, picu...","[[currently, RB], [completing, VBG], [ICU, NNP..."


In [ ]:
#export
def append_phrase_column(data, feature_names, tdm, window_size = 6, include_input_in_tfidf = False):
    """
    Extracts the most expressive phrase of length `window_size` from each sentence and appends them to the input dataframe in a new 'phrase' column. 
    Sentences without phrases of sufficient length are set to None.
    """
    adj_adv_pos_list = ["JJ","JJR", "JJS", "RB", "RBR", "RBS"]
    top_phrases = []
    
    if not include_input_in_tfidf:
        token_averages = get_term_max(tdm)
        
    # Iterate sentences
    for ix, sent in data.iterrows():
        # Iterate phrases (sub-sentences of length window_size)
        phrase_scores = []
        for p in range(len(sent.tokens) - window_size + 1):
            window = slice(p, p + window_size)
            phrase = sent.tokens[window]
            phrase_pos = sent.pos_tags[window]

            weight = 1 + abs(TextBlob(" ".join(phrase)).sentiment.polarity)
            score = 0

            for i, token in enumerate(phrase):
                # Skip tokens not in feature_names
                if token not in list(feature_names.keys()):
                    continue

                pos = phrase_pos[i][1]
                token_ix = feature_names[token]

                # Token score comes from TF-IDf matrix if include_input_in_tfidf is set, otherwise, use tokens averages
                token_score = tdm[token_ix, sent.doc_id] if include_input_in_tfidf else token_averages[token_ix];

                # Scale token_score by 3x if the token is an adjective or adverb
                score += (token_score * 3) if pos in adj_adv_pos_list else token_score

            phrase_scores.append((score*weight, phrase))

        # Select the phrase with the highest score, use None if there are no phrases
        phrase_scores.sort(key=lambda sent: sent[0], reverse=True)
        top_phrase = phrase_scores[0][1] if len(phrase_scores) > 0 else None
        top_phrases.append(top_phrase) 
    
    # Append a phrase column to the 'data' dataframe
    data['phrase'] = top_phrases
    return data

In [ ]:
#export
def get_doc_word_vectors_tfidf(doc_top_phrases, dictionary, tfidf_dense):
    "Uses the TF-IDF matrix to create a list of lists corresponding to a word vectors for the top phrases of each sentence in each document"
    return get_phrase_vec_tfidf(doc_top_phrases, dictionary, tfidf_dense)

def get_doc_word_vectors_svd(doc_top_phrases, dictionary, tfidf_dense, dimensions = 2):
    "Decomposes the TF-IDF matrix via SVD and uses the result to create a list of lists corresponding to a word vectors for the top phrases of each sentence in each document"
    svd =  TruncatedSVD(n_components = dimensions, random_state = 42)
    tfidf_transf = svd.fit_transform(tfidf_dense)
    return get_phrase_vec_tfidf(doc_top_phrases, dictionary, tfidf_transf)

def get_doc_word_vectors_umap(doc_top_phrases, dictionary, tfidf_dense, umap_neighbors = 15, dimensions = 2):
    "Transforms the TF-IDF matrix via UMAP and uses the result to create a list of lists corresponding to a word vectors for the top phrases of each sentence in each document"
    reducer = umap.UMAP(n_neighbors=umap_neighbors, min_dist=.1, metric='cosine', random_state=42, n_components = dimensions)
    embed = reducer.fit_transform(tfidf_dense)
    return get_phrase_vec_tfidf(doc_top_phrases, dictionary, embed)

def get_doc_word_vectors_pretrained(doc_top_phrases, path_to_w2v_bin_file):
    "Uses pretrained Word2Vec embeddings to create a list of lists corresponding to a word vectors for the top phrases of each sentence in each document"
    assert os.path.exists(path_to_w2v_bin_file), f'The file {path_to_w2v_bin_file} does not exist'
    model = w2v_pretrained(path_to_w2v_bin_file)
    return get_phrase_vec_w2v(doc_top_phrases, model)

def get_doc_word_vectors_local(doc_top_phrases, raw_docs, my_docs):
    "Creates Word2Vec embeddings from the input corpus and uses them to create a list of lists corresponding to a word vectors for the top phrases of each sentence in each document"
    model = w2v_from_corpus(raw_docs, my_docs)
    return get_phrase_vec_w2v(doc_top_phrases, model)

In [ ]:
#export
def filter_sentences(doc_phrase_vecs, doc_top_phrases):
    "Filter the sentences, removing any that contain zero 'top phrases'"
    just_phrase_vecs = []
    just_phrase_ids = []
    just_phrase_text = []

    # Getting all phrases which have 
    # Iterate documents
    for doc_ix, phrase_vecs in enumerate(doc_phrase_vecs):
        # Iterate sentences (each sentence has a phrase vector)
        for phrase_ix, phrase_vec in enumerate(phrase_vecs):
            phrase_id = f"doc.{doc_ix}.sent.{phrase_ix}"
            this_text = doc_top_phrases[doc_ix][phrase_ix]

            # Check phrase_vec is not zero or an array of zeros, these would imply the sentence has no "top phrases"
            if isinstance(phrase_vec, np.ndarray) and any(np.zeros(len(phrase_vec)) != phrase_vec):
                just_phrase_vecs.append(list(phrase_vec))
                just_phrase_ids.append(phrase_id)
                just_phrase_text.append(this_text[1])
    return just_phrase_vecs, just_phrase_ids, just_phrase_text

In [ ]:
#export 
def get_optimal_k(just_phrase_vecs, save_chart = False, chart_file = "KmeansSilhouette.png"):
    """
    Clusters the top phrase vectors for a range of k values and plots the silhoute coefficients 
    (this is a function of mean intra-cluster distance and mean nearest-cluster distance). 
    Returns the optimal k-value (highest silhoute coefficient)
    """
    max_k = min(len(just_phrase_vecs), 100)
    k_range = range(2, max_k)
    score = [(silhouette_score(just_phrase_vecs, KMeans(i).fit(just_phrase_vecs).predict(just_phrase_vecs))) for i in k_range]
    fig = plt.plot(k_range, score)
    if save_chart:
        plt.savefig(chart_file, dpi=300)
    optimal_k = k_range[np.argmax(score)]
    return optimal_k

def get_cluster_assignments_kmeans(k, just_phrase_vecs):
    "Use K-means algorithm to cluster phrase vectors"
    kmeans = KMeans(n_clusters=k, random_state=42).fit(just_phrase_vecs)
    cluster_assignments = kmeans.predict(just_phrase_vecs)
    #TODO: this is never used before being overwritten later. Remove?
    dist = pairwise_distances(just_phrase_vecs, metric='euclidean') 
    return cluster_assignments, dist

In [ ]:
#export
def get_linkage_matrix(just_phrase_vecs, dist_metric):
    "Creates a linkage matrix by calculating distance between phrase vectors"
    if dist_metric == "cosine":
        dist = 1 - cosine_similarity(just_phrase_vecs)
    else:
        dist = pairwise_distances(just_phrase_vecs, metric=dist_metric)
    return ward(dist)

def get_optimal_height(just_phrase_vecs, linkage_matrix, save_chart = False, chart_file = "HACSilhouette.png"):
    """
    Clusters the top phrase vectors and plots the silhoute coefficients for a range of dendrograph heights. 
    Returns the optimal height value (highest silhoute coefficient)
    """
    max_h = min(len(just_phrase_vecs), 100)
    # QUESTION: What is the limiting factor for the min/max in this range?
#     h_range = range(30, max_h) 
    h_range = range(1,10) 
    h_score = [(silhouette_score(just_phrase_vecs, [x[0] for x in cut_tree(linkage_matrix, height=i)] )) for i in h_range]

    fig = plt.plot(h_range, h_score)

    if save_chart:
        plt.savefig(chart_file, dpi=300)
    optimal_h = h_range[np.argmax(h_score)]
    return optimal_h

def get_cluster_assignments_hac(linkage_matrix, height):
    "Use Hierarchical Agglomerative Clustering to cluster phrase vectors"
    return [x[0] for x in cut_tree(linkage_matrix, height=height)]

In [ ]:
#export
def visualize_umap(just_phrase_ids, cluster_assignments, just_phrase_text, dist, umap_neighbors = 15, save_chart = False, chart_file = "UMAP.html"):
    "Visualize the clusters using UMAP"
    reducer = umap.UMAP(n_neighbors=umap_neighbors, min_dist=.1, metric='cosine', random_state=42)
    embedding = reducer.fit_transform(dist)
    df = pd.DataFrame(dict(label=just_phrase_ids, UMAP_cluster=cluster_assignments, phrase=just_phrase_text, x=embedding[:, 0], y=embedding[:, 1]))
    fig = px.scatter(df, x="x", y="y", hover_name="label", color="UMAP_cluster", hover_data=["phrase", "label","UMAP_cluster"], color_continuous_scale='rainbow')
    fig.show()
    if save_chart:
        plotly.offline.plot(fig, filename=chart_file)
        
def visualize_mds(just_phrase_ids, cluster_assignments, just_phrase_text, dist, save_chart = False, chart_file = "MDS.html"):
    "Visualize the clusters using Multi-Dimensional Scaling (MDS)"
    mds = MDS(n_components=2, dissimilarity="precomputed", random_state=42)
    pos = mds.fit_transform(dist)
    xs, ys = pos[:, 0], pos[:, 1]
    df = pd.DataFrame(dict(x=xs, y=ys, MDS_cluster=cluster_assignments, label=just_phrase_ids, phrase=just_phrase_text)) 
    fig = px.scatter(df, x="x", y="y", hover_name="label", color="MDS_cluster", hover_data=["phrase", "label","MDS_cluster"], color_continuous_scale='rainbow')
    fig.show()
    if save_chart:
        plotly.offline.plot(fig, filename=chart_file)
        
def visualize_svd(just_phrase_ids, cluster_assignments, just_phrase_text, dist, save_chart = False, chart_file = "SVD.html"):
    "Visualize the clusters using Singular Value Decomposition (SVD)"
    svd2d =  TruncatedSVD(n_components = 2, random_state = 42).fit_transform(dist)
    df = pd.DataFrame(dict(label=just_phrase_ids, SVD_cluster=cluster_assignments, phrase=just_phrase_text, x=svd2d[:, 0], y=svd2d[:, 1])) 
    fig = px.scatter(df, x="x", y="y", hover_name="label", color="SVD_cluster", hover_data=["phrase", "label","SVD_cluster"], color_continuous_scale='rainbow')
    fig.show()
    if save_chart:
        plotly.offline.plot(fig, filename=chart_file)

In [ ]:
#export
def get_cluster_topics(cluster_assignments, just_phrase_ids, my_docs):
    "Gets a list of the main topics for each cluster"
    cluster_topics = []
    for c in set(cluster_assignments):
        sent_coords = get_cluster_sent_coords(c, just_phrase_ids, cluster_assignments)

        ## sent_coords has each element formatted as [doc number, sentence number]
        cluster_terms = []
        for doc_id, sent_id in sent_coords:
            sent = my_docs[doc_id][sent_id]
            # This isn't used. Can we remove?
#             pos = my_docs_pos[doc_id][sent_id] 
            cluster_terms.append(sent)

        cluster_topics.append(cluster_terms)
        
    main_cluster_topics = []
    for doc_topics in cluster_topics:
        #Latent Dirichlet Allocation implementation with Gensim
        dictionary = corpora.Dictionary(doc_topics)
        corpus = [dictionary.doc2bow(text) for text in doc_topics]
        lda = models.LdaModel(corpus, num_topics=1, id2word=dictionary)
        topics_matrix = lda.show_topics(formatted=False, num_words=10)

        topics_ary = list(np.array(topics_matrix[0][1])[:,0])
        main_cluster_topics.append(topics_ary)
    return main_cluster_topics

In [ ]:
#export
def write_output_to_disk(main_cluster_topics, just_phrase_ids, cluster_assignments, raw_sentences, 
                         doc_top_phrases, file_list, file_name = "TopicClusterResults.txt"):
    "Creates a CSV file listing cluster_id/doc_id/doc_name/sentence_id/sentence_text/sentences_phrases"
    with open(file_name, "w") as outfile:
        for c, topics in enumerate(main_cluster_topics):
            outfile.write("Cluster\tDocument.Num\tDocument.Name\tSentence.Num\tSentence.Text\nPhrase.Text")
            sent_coords = get_cluster_sent_coords(c, just_phrase_ids, cluster_assignments)
            outfile.write(f"Cluster {c} Keywords: {', '.join(topics)}\n")

            ## sent_coords has each element formatted as [doc number, sentence number]
            for doc_id, sent_id in sent_coords:
                if doc_id >= 0 and sent_id >= 0:
                    outfile.write(f"{c}\t{doc_id}\t{file_list[doc_id]}\t{sent_id}\t{raw_sentences[doc_id][sent_id]}\t{doc_top_phrases[doc_id][sent_id]}\n")
                else:
                    #QUESTION: What are we trying to do here? When would this be hit?
                    outfile.write(f"{c}\t{doc_id, sent_id}\t{doc_id, sent_id}\t{doc_id, sent_id}\t{doc_id, sent_id}\t{doc_id, sent_id}\n")

In [ ]:
#export
def evaluate(just_phrase_ids, cluster_assignments, just_phrase_text, 
             gold_file = "data/GOLD_Expert_2019.03.12_TestCorpus_AMY.txt", output_file = "output/EvaluationResults.txt"):
    "Evaluate precision, recall, and F1 against a gold standard dataset. Write results to a file"
    
    df = pd.DataFrame(dict(label=just_phrase_ids, cluster=cluster_assignments, phrase=just_phrase_text))
    
    # Process the gold standard file into a list of IDs and classification groups
    with open(gold_file) as file:
        gold_list = file.read().strip().split('\n')
    gold_array = np.array([x.split('\t') for x in gold_list])
    ids, group = gold_array.T    
    baseline = list(np.unique(group)) # This is the list of classes ['Confidence', 'Overwhelmed', 'Supportive Environment', 'System Issues']

    # Dictionary of IDs corresponding to each class
    gold_clusters = {}
    for clazz in baseline:
        # List of IDs (doc.#.sent.#) corresponding to a given class
        gold_clusters[clazz] = gold_array[gold_array[:,1] == clazz][:,0]

    #TODO: This can be combined with the for loop above
    closest_to_gold = {}
    for b in baseline:
        max_overlap = 0
        max_overlap_cluster = -1
        cluster_ids = list(set(cluster_assignments))
        just_phrase_ids

        for c in cluster_ids:
            overlap = len(set(gold_clusters[b]).intersection(set(df.label[df.cluster==c])))
            if overlap > max_overlap:
                max_overlap = overlap
                max_overlap_cluster = c
        closest_to_gold[b] = max_overlap_cluster

    #TODO: I think this could also be combined with the above FOR loop
    # Write results to a file
    with open(output_file, "w") as eout:
        eout.write("Gold Concept\tGold Concept Members\tClosest Cluster Num\tClosest Cluster Members\tTP\tFP\tFN\tP\tR\tF1\n")
        for g in gold_clusters.keys():
            ## We only want to do these calculations on sentences that are in the gold file.
            ## Get closest cluster list of sentences that are ONLY in gold
            closest = set(df.label[df.cluster==closest_to_gold[g]]).intersection(set(ids))
            gold = set(gold_clusters[g])
            TP = len(gold.intersection(closest))
            FP = len(closest - gold)
            FN = len(gold - closest)
            P = round(TP/(TP+FP), 3) if (TP+FP) > 0 else float("Nan")
            R = round(TP/(TP+FN), 3) if (TP+FN) > 0 else float("Nan")
            F1 = round(2*((P*R)/(P+R)), 3) if (P+R) > 0 else float("Nan")
            eout.write(f"{g}\t{gold}\t{closest_to_gold[g]}\t{closest}\t{TP}\t{FP}\t{FN}\t{P}\t{R}\t{F1}\n")

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted Core.ipynb.
Converted helpers.ipynb.
Converted index.ipynb.
Converted nlp_helpers.ipynb.
Converted preprocessing.ipynb.
Converted Sandbox.ipynb.
